In [1]:
# ============================================
# [제출 전용] 공행성(pair) 판별 + 다음달(value) 예측
# - validation 없음 (제출 무제한 가정)
# - "처음 0.34 나오던" 상관 기반 구조를 최대한 유지
# - 주석 한국어 상세
#
# 핵심 개선 포인트(이번 버전):
# 1) pair 선택 시 PROB_TAU를 써도 "항상 K개" 제출하도록 backfill(빈자리 채움)
#    -> tau가 빡세서 K를 못 채우면 FN(놓침) 증가로 점수 급락하는 현상 방지
# 2) 회귀 예측은 XGBRegressor로 하고, follower 자체 MA3 baseline 블렌딩 옵션 유지
# ============================================

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
from xgboost import XGBClassifier, XGBRegressor


# =========================================================
# 0) ✅ 실험/제출용 하이퍼파라미터 (여기만 바꿔가며 제출)
# =========================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# (1) lag(선행-후행 시차) 후보 범위: 1~MAX_LAG
MAX_LAG = 7

# (2) pseudo-label(약지도) 생성 시 positive로 찍는 상관 임계값
#     - 너무 높으면 pos가 너무 적어져서 classifier가 약해짐
#     - 너무 낮으면 pos가 너무 많아져서 classifier가 구분을 못함
PAIR_LABEL_CORR_THRESHOLD = 0.38  # 0.34~0.40 sweep 추천

# (3) classifier 학습용: 너무 0이 많은 품목 제외(희소성 노이즈 줄이기)
PAIR_MIN_NONZERO = 8

# (4) 전체 pair 스코어링용: min_nonzero를 완화해서 recall 확보
SCORE_MIN_NONZERO = 2

# (5) negative sampling 비율 (pos:neg = 1:NEG_POS_RATIO)
NEG_POS_RATIO = 2.0

# (6) ✅ 최종 제출 pair 개수 (가장 중요)
PAIR_TOP_K = 3500  # 2500, 3000, 3500, 4000 등 sweep

# (7) ✅ classifier 확률 컷 (FP 줄이기 위한 옵션)
#     - 이번 버전은 "tau를 걸어도 K를 반드시 채우도록(backfill)" 설계했으므로
#       tau를 써도 FN 폭증 위험이 훨씬 줄어듦
USE_PROB_THRESHOLD = True
PROB_TAU = 0.28  # 0.22~0.35 sweep 추천

# (8) (선택) corr_stability 필터: 우연 상관 제거
#     - 너무 빡세면 FN 늘어서 점수 떨어질 수 있음
USE_STABILITY_FILTER = False
STAB_MAX = 0.20  # 켜면 0.15~0.30 sweep

# (9) 회귀 예측 안정화: follower 자체 baseline(ma3) 블렌딩
USE_BASELINE_BLEND = True
BLEND_ALPHA = 0.95  # 0.90~0.98 sweep 추천 (너 결과상 0.95가 괜찮은 편)

# (10) 회귀 모델 파라미터 (너무 자주 건드릴 필요는 없음)
REG_N_ESTIMATORS = 600
REG_MAX_DEPTH = 4
REG_LR = 0.05

# (11) 분류기 파라미터 (처음 스타일 유지)
CLF_N_ESTIMATORS = 200
CLF_MAX_DEPTH = 4
CLF_LR = 0.08


# =========================================================
# 1) 유틸: 안전 상관계수 계산
# =========================================================
def safe_corr(a: np.ndarray, b: np.ndarray) -> float:
    """
    Pearson 상관계수를 안전하게 계산
    - NaN 제거
    - 샘플 수 너무 적으면 0
    - 분산 0이면 0
    """
    mask = (~np.isnan(a)) & (~np.isnan(b))
    if mask.sum() < 3:
        return 0.0
    aa, bb = a[mask], b[mask]
    if np.std(aa) == 0 or np.std(bb) == 0:
        return 0.0
    return float(np.corrcoef(aa, bb)[0, 1])


# =========================================================
# 2) 데이터 로딩: 월별 value pivot 생성
# =========================================================
def load_pivot(train_path="train.csv") -> pd.DataFrame:
    """
    train.csv -> item_id x 월(ym) pivot 생성
    - 월별 합계(value sum)가 문제 정의에 맞음
    - 결측 월은 0으로 채움
    """
    df = pd.read_csv(train_path)

    monthly = (
        df.groupby(["item_id", "year", "month"], as_index=False)["value"]
          .sum()
    )

    monthly["ym"] = pd.to_datetime(
        monthly["year"].astype(str) + "-" +
        monthly["month"].astype(str).str.zfill(2) + "-01"
    )

    pivot = monthly.pivot(index="item_id", columns="ym", values="value")
    pivot = pivot.fillna(0).sort_index(axis=1)

    print("[pivot]", pivot.shape,
          "| months:", pivot.columns[0].strftime("%Y-%m"), "->", pivot.columns[-1].strftime("%Y-%m"))
    return pivot


# =========================================================
# 3) pair의 lag-corr 통계 피처 계산 (핵심)
# =========================================================
def calc_pair_stats(a: np.ndarray, b: np.ndarray, max_lag: int) -> dict:
    """
    a가 선행, b가 후행이라고 가정하고
    lag=1..max_lag에 대해 corr(a[t], b[t+lag]) 계산.

    반환:
    - max_corr: 가장 큰(절대값 기준) 상관
    - best_lag: 그때의 lag
    - second_corr: 2등 상관
    - corr_stability: |best - second| (작을수록 '비슷한 lag들에서 안정적')
    - corr_mean/std/abs_mean: lag별 상관 통계
    """
    n = len(a)
    lag_corrs = []

    best_corr = 0.0
    second_corr = 0.0
    best_lag = None

    for lag in range(1, max_lag + 1):
        if n <= lag:
            lag_corrs.append(0.0)
            continue

        c = safe_corr(a[:-lag], b[lag:])
        lag_corrs.append(c)

        # 절대값 기준으로 1등/2등 갱신
        if abs(c) > abs(best_corr):
            second_corr = best_corr
            best_corr = c
            best_lag = lag
        elif abs(c) > abs(second_corr):
            second_corr = c

    if best_lag is None:
        best_lag = 1

    lag_corrs = np.array(lag_corrs, dtype=float)

    return {
        "max_corr": float(best_corr),
        "best_lag": int(best_lag),
        "second_corr": float(second_corr),
        "corr_stability": float(abs(best_corr - second_corr)),
        "corr_mean": float(np.mean(lag_corrs)),
        "corr_std": float(np.std(lag_corrs)),
        "corr_abs_mean": float(np.mean(np.abs(lag_corrs))),
    }


# =========================================================
# 4) classifier 학습용 pseudo-label 데이터 생성
# =========================================================
def build_pair_feature_matrix(
    pivot: pd.DataFrame,
    max_lag: int,
    min_nonzero: int,
    corr_threshold_for_label: float,
    neg_pos_ratio: float
) -> pd.DataFrame:
    """
    정답 pair가 없으니, 상관이 큰 pair를 positive로 가정해 pseudo-label 생성.
    - label=1: abs(max_corr) >= corr_threshold_for_label
    - label=0: 그 외
    """
    items = pivot.index.to_list()

    rows_pos, rows_neg = [], []

    for leader in tqdm(items, desc="build_pair_features(clf)"):
        a = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(a) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            b = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(b) < min_nonzero:
                continue

            feats = calc_pair_stats(a, b, max_lag)
            label = 1 if abs(feats["max_corr"]) >= corr_threshold_for_label else 0

            row = {
                "leading_item_id": leader,
                "following_item_id": follower,
                **feats,
                "nonzero_a": int(np.count_nonzero(a)),
                "nonzero_b": int(np.count_nonzero(b)),
                "sum_a": float(a.sum()),
                "sum_b": float(b.sum()),
                "label": int(label),
            }

            if label == 1:
                rows_pos.append(row)
            else:
                rows_neg.append(row)

    df_pos = pd.DataFrame(rows_pos)
    df_neg = pd.DataFrame(rows_neg)
    print("[clf pseudo-label] pos:", df_pos.shape, "| neg:", df_neg.shape)

    if df_pos.empty:
        return pd.DataFrame()

    # negative sampling (학습 속도/불균형 조절)
    n_pos = len(df_pos)
    n_neg_keep = int(neg_pos_ratio * n_pos)
    if len(df_neg) > n_neg_keep:
        df_neg = df_neg.sample(n_neg_keep, random_state=RANDOM_SEED)

    df_all = pd.concat([df_pos, df_neg], axis=0).reset_index(drop=True)
    print("[clf train set]", df_all.shape)
    return df_all


def train_pair_classifier(df_pairs: pd.DataFrame):
    """
    XGBClassifier로 pair가 공행성일 확률(clf_prob)을 예측하도록 학습.
    """
    feature_cols = [
        "max_corr", "best_lag", "second_corr",
        "corr_stability", "corr_mean", "corr_std", "corr_abs_mean",
        "nonzero_a", "nonzero_b", "sum_a", "sum_b"
    ]

    df = df_pairs.copy()
    df[feature_cols] = df[feature_cols].replace([np.inf, -np.inf], 0).fillna(0)

    X = df[feature_cols].values
    y = df["label"].values

    clf = XGBClassifier(
        n_estimators=CLF_N_ESTIMATORS,
        max_depth=CLF_MAX_DEPTH,
        learning_rate=CLF_LR,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_alpha=0.5,
        reg_lambda=1.0,
        min_child_weight=3,
        gamma=0.2,
        random_state=RANDOM_SEED,
        n_jobs=-1,
        eval_metric="logloss"
    )
    clf.fit(X, y)
    return clf, feature_cols


# =========================================================
# 5) 전체 pair scoring: clf_prob 계산 + stats 저장
# =========================================================
def score_all_pairs(pivot: pd.DataFrame, clf, clf_feature_cols, max_lag: int, min_nonzero: int) -> pd.DataFrame:
    """
    모든 (leader -> follower) 쌍에 대해:
    - lag-corr 통계 피처 계산
    - classifier 확률 clf_prob 계산
    """
    items = pivot.index.to_list()
    rows = []

    for leader in tqdm(items, desc="score_all_pairs"):
        a = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(a) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            b = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(b) < min_nonzero:
                continue

            feats = calc_pair_stats(a, b, max_lag)
            extra = {
                "nonzero_a": int(np.count_nonzero(a)),
                "nonzero_b": int(np.count_nonzero(b)),
                "sum_a": float(a.sum()),
                "sum_b": float(b.sum()),
            }

            x_vec = np.array([[{**feats, **extra}[c] for c in clf_feature_cols]], dtype=float)
            prob = float(clf.predict_proba(x_vec)[0, 1])

            rows.append({
                "leading_item_id": leader,
                "following_item_id": follower,
                "clf_prob": prob,
                **feats
            })

    df = pd.DataFrame(rows)
    df = df.sort_values("clf_prob", ascending=False).reset_index(drop=True)
    return df


# =========================================================
# 6) 회귀 데이터셋 생성/학습
# =========================================================
REG_FEATURE_COLS = [
    "b_t", "b_t_1", "b_t_2",
    "b_ma3", "b_change",
    "a_t_lag", "a_t_lag_1",
    "a_ma3", "a_change",
    "ab_value_ratio",
    "max_corr", "best_lag", "corr_stability",
]

def build_reg_dataset(pivot: pd.DataFrame, pairs: pd.DataFrame, target_end_idx: int) -> pd.DataFrame:
    """
    각 pair(A->B)에 대해 여러 시점 t를 만들어 (t+1)을 예측하도록 학습 row 생성.

    target_end_idx:
    - 학습 타깃의 마지막 인덱스(포함)
    - 마지막 관측 월이 last_idx면, (t+1)이 last_idx까지 학습 가능하므로
      target_end_idx는 보통 last_idx-1로 둠(아래 run_submission 참고)
    """
    rows = []

    for r in tqdm(pairs.itertuples(index=False), desc="build_reg_dataset"):
        leader = r.leading_item_id
        follower = r.following_item_id
        lag = int(r.best_lag)

        a = pivot.loc[leader].values.astype(float)
        b = pivot.loc[follower].values.astype(float)

        # t+1이 타깃이므로 t는 target_end_idx-1까지 가능
        # 또한 b_t_2(=t-2), a_t_lag_1(=t-lag-1) 등이 필요하므로 최소 인덱스가 필요
        for t in range(lag + 2, target_end_idx):
            target_idx = t + 1
            if target_idx > target_end_idx:
                continue

            # follower 최근 값들
            b_t = b[t]
            b_t_1 = b[t - 1]
            b_t_2 = b[t - 2]
            b_ma3 = (b_t + b_t_1 + b_t_2) / 3.0
            b_change = (b_t - b_t_1) / (b_t_1 + 1.0)

            # leader lag 적용 값들
            a_t_lag = a[t - lag]
            a_t_lag_1 = a[t - lag - 1]
            a_change = (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1.0)

            # leader도 3개월 평균(가능하면)
            if (t - lag - 2) >= 0:
                a_ma3 = (a_t_lag + a_t_lag_1 + a[t - lag - 2]) / 3.0
            else:
                a_ma3 = (a_t_lag + a_t_lag_1) / 2.0

            # ratio
            ab_ratio = b_t / (a_t_lag + 1.0)

            y = b[target_idx]

            rows.append({
                "leading_item_id": leader,
                "following_item_id": follower,

                "b_t": b_t,
                "b_t_1": b_t_1,
                "b_t_2": b_t_2,
                "b_ma3": b_ma3,
                "b_change": b_change,

                "a_t_lag": a_t_lag,
                "a_t_lag_1": a_t_lag_1,
                "a_ma3": a_ma3,
                "a_change": a_change,

                "ab_value_ratio": ab_ratio,

                "max_corr": float(r.max_corr),
                "best_lag": lag,
                "corr_stability": float(r.corr_stability),

                "target": y
            })

    df = pd.DataFrame(rows)
    print("[reg train set]", df.shape)
    return df


def train_regressor(df_train: pd.DataFrame):
    """
    XGBRegressor 학습
    """
    df = df_train.replace([np.inf, -np.inf], 0).fillna(0)

    X = df[REG_FEATURE_COLS].values
    y = df["target"].values

    reg = XGBRegressor(
        n_estimators=REG_N_ESTIMATORS,
        max_depth=REG_MAX_DEPTH,
        learning_rate=REG_LR,
        subsample=0.85,
        colsample_bytree=0.85,
        min_child_weight=5,
        gamma=0.2,
        reg_alpha=0.5,
        reg_lambda=1.0,
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    reg.fit(X, y)
    return reg


def follower_ma3(b: np.ndarray, t: int) -> float:
    """
    follower 자체 baseline: 최근 3개월 평균
    """
    if t < 2:
        return float(b[t])
    return float((b[t] + b[t-1] + b[t-2]) / 3.0)


# =========================================================
# 7) ✅ 제출 파일 생성 (submission-only)
# =========================================================
def run_submission(train_path="train.csv", out_path="submission.csv"):
    """
    전체 파이프라인(제출 전용)

    1) pivot 생성
    2) pseudo-label 생성 -> pair classifier 학습
    3) 모든 pair clf_prob 스코어링
    4) pair 선택: (옵션) stability 필터 + (옵션) tau 필터
       - 핵심: tau로 거른 뒤에도 "항상 K개" 채우도록 backfill
    5) 회귀 학습
    6) 마지막 월 기준 다음달(value) 예측 -> CSV 저장
    """
    pivot = load_pivot(train_path)
    months = list(pivot.columns)

    last_idx = len(months) - 1     # 마지막 관측 월 인덱스 (예: 2025-07)
    target_end_idx = last_idx - 1  # 회귀 학습에서 target은 t+1 이므로 마지막 월은 target으로 못 씀

    # ---- (A) pseudo-label 생성 -> classifier 학습
    df_pairs_label = build_pair_feature_matrix(
        pivot=pivot,
        max_lag=MAX_LAG,
        min_nonzero=PAIR_MIN_NONZERO,
        corr_threshold_for_label=PAIR_LABEL_CORR_THRESHOLD,
        neg_pos_ratio=NEG_POS_RATIO
    )
    if df_pairs_label.empty:
        raise RuntimeError("pseudo-label pos가 0개입니다. PAIR_LABEL_CORR_THRESHOLD를 낮춰보세요.")

    clf, clf_feature_cols = train_pair_classifier(df_pairs_label)

    # ---- (B) 전체 pair 스코어링
    pairs_all = score_all_pairs(
        pivot=pivot,
        clf=clf,
        clf_feature_cols=clf_feature_cols,
        max_lag=MAX_LAG,
        min_nonzero=SCORE_MIN_NONZERO
    )

    # ---- (C) 제출 pair 선택
    # 0) 기본은 전체에서 prob 높은 순
    pairs_base = pairs_all.copy()

    # 1) (선택) stability 필터
    if USE_STABILITY_FILTER:
        pairs_base = pairs_base[pairs_base["corr_stability"] <= STAB_MAX]

    # 2) (선택) tau 필터를 먼저 적용하되, K를 못 채우면 backfill
    if USE_PROB_THRESHOLD:
        pairs_tau = pairs_base[pairs_base["clf_prob"] >= PROB_TAU].copy()

        if len(pairs_tau) >= PAIR_TOP_K:
            # tau 통과분이 충분하면 그 안에서 topK
            pairs_sel = pairs_tau.sort_values("clf_prob", ascending=False).head(PAIR_TOP_K)
        else:
            # tau 통과분이 부족하면, 남은 자리는 전체(prob 높은 순)에서 추가로 채움
            need = PAIR_TOP_K - len(pairs_tau)
            # pairs_base 기준으로 채우는 게 일관성 있음(=stability 필터 반영됨)
            rest = pairs_base.loc[~pairs_base.index.isin(pairs_tau.index)] \
                             .sort_values("clf_prob", ascending=False) \
                             .head(need)
            pairs_sel = pd.concat([pairs_tau, rest], axis=0)
    else:
        # tau를 안 쓰면 그냥 prob 상위 K
        pairs_sel = pairs_base.sort_values("clf_prob", ascending=False).head(PAIR_TOP_K)

    pairs_sel = pairs_sel.reset_index(drop=True)

    print(
        f"[pair 선택] 실제 제출 {len(pairs_sel)}개 | 목표K={PAIR_TOP_K} | "
        f"use_tau={USE_PROB_THRESHOLD} tau={PROB_TAU} | "
        f"use_stab={USE_STABILITY_FILTER} stab={STAB_MAX if USE_STABILITY_FILTER else 'OFF'}"
    )

    if len(pairs_sel) == 0:
        raise RuntimeError("선택된 pair가 0개입니다. PROB_TAU를 낮추거나 필터를 끄세요.")

    # ---- (D) 회귀 학습
    df_reg = build_reg_dataset(pivot, pairs_sel, target_end_idx=target_end_idx)
    if len(df_reg) == 0:
        raise RuntimeError("회귀 학습 데이터가 0개입니다. PAIR_TOP_K를 늘리거나 필터를 완화하세요.")

    reg = train_regressor(df_reg)

    # ---- (E) 마지막 관측 월(last_idx)을 기준으로 다음달 예측
    sub_rows = []

    for r in tqdm(pairs_sel.itertuples(index=False), desc="make_submission"):
        leader = r.leading_item_id
        follower = r.following_item_id
        lag = int(r.best_lag)

        a = pivot.loc[leader].values.astype(float)
        b = pivot.loc[follower].values.astype(float)

        t = last_idx  # 마지막 관측 월을 t로 보고 (t+1)을 예측

        # 필요한 과거값 체크
        if t - 2 < 0:
            continue
        if t - lag - 1 < 0:
            continue

        # follower
        b_t = b[t]
        b_t_1 = b[t - 1]
        b_t_2 = b[t - 2]
        b_ma3 = (b_t + b_t_1 + b_t_2) / 3.0
        b_change = (b_t - b_t_1) / (b_t_1 + 1.0)

        # leader (lag 적용)
        a_t_lag = a[t - lag]
        a_t_lag_1 = a[t - lag - 1]
        a_change = (a_t_lag - a_t_lag_1) / (a_t_lag_1 + 1.0)

        if (t - lag - 2) >= 0:
            a_ma3 = (a_t_lag + a_t_lag_1 + a[t - lag - 2]) / 3.0
        else:
            a_ma3 = (a_t_lag + a_t_lag_1) / 2.0

        ab_ratio = b_t / (a_t_lag + 1.0)

        feat_row = {
            "b_t": b_t,
            "b_t_1": b_t_1,
            "b_t_2": b_t_2,
            "b_ma3": b_ma3,
            "b_change": b_change,
            "a_t_lag": a_t_lag,
            "a_t_lag_1": a_t_lag_1,
            "a_ma3": a_ma3,
            "a_change": a_change,
            "ab_value_ratio": ab_ratio,
            "max_corr": float(r.max_corr),
            "best_lag": int(r.best_lag),
            "corr_stability": float(r.corr_stability),
        }

        x_row = np.array([[feat_row[c] for c in REG_FEATURE_COLS]], dtype=float)
        pred = float(reg.predict(x_row)[0])
        pred = max(pred, 0.0)

        # (선택) follower baseline과 블렌딩
        if USE_BASELINE_BLEND:
            base = follower_ma3(b, t)
            pred = BLEND_ALPHA * pred + (1.0 - BLEND_ALPHA) * base
            pred = max(pred, 0.0)

        sub_rows.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": int(round(pred))
        })

    df_sub = pd.DataFrame(sub_rows).drop_duplicates(["leading_item_id", "following_item_id"])
    df_sub.to_csv(out_path, index=False)
    print(f"\n✅ 저장 완료: {out_path} | shape={df_sub.shape}")


# =========================================================
# 8) 실행부: OUT 파일명만 바꿔서 제출 반복
# =========================================================
if __name__ == "__main__":
    # ✅ 파일명만 바꿔가며 제출
    OUT = "firststyle_k3500_tau028_ble095_backfill.csv"
    run_submission("train.csv", OUT)


[pivot] (100, 43) | months: 2022-01 -> 2025-07


build_pair_features(clf): 100%|██████████| 100/100 [00:18<00:00,  5.49it/s]


[clf pseudo-label] pos: (1991, 14) | neg: (6565, 14)
[clf train set] (5973, 14)


score_all_pairs: 100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


[pair 선택] 실제 제출 3500개 | 목표K=3500 | use_tau=True tau=0.28 | use_stab=False stab=OFF


build_reg_dataset: 3500it [00:01, 2821.95it/s]


[reg train set] (123556, 16)


make_submission: 3500it [00:07, 455.52it/s]


✅ 저장 완료: firststyle_k3500_tau028_ble095_backfill.csv | shape=(3500, 3)
